# Setup

In [2]:
# Image from Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install --quiet -U --no-cache-dir gdown --pre
!pip install --quiet nlp-id
!pip install -qq transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.4 MB/s eta 0:00:00


In [4]:
from nlp_id.stopword import StopWord
from nlp_id.postag import PosTag
from nlp_id.lemmatizer import Lemmatizer

from imblearn.over_sampling import ADASYN 

from xgboost.sklearn import XGBClassifier

import string
import re
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K

import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')

import seaborn as sns # untuk visualisasi
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

import torch
import torch.nn.functional as F
import torch.nn as nn

from transformers import AutoTokenizer, AutoModel, get_constant_schedule_with_warmup
from transformers import TrainingArguments, Trainer, AutoConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# potongan kode di bawah adalah agar gambar graph yang 
# dihasilkan mempunyai kualitas yang cukup baik
sns.set(rc={"figure.dpi":100, 'savefig.dpi':300})
sns.set_context('notebook')
sns.set_style("ticks")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Ibrohim-Budi Data

In [7]:
abusive_lexicon = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/abusive.csv')
kamus_alay = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/new_kamusalay.csv', encoding='latin-1')

test_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/test_data.csv', encoding='latin-1')
synthetic_test_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/synthetic_test_data_5x.csv', encoding='latin-1')
remaining_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/remaining_data.csv', encoding='latin-1')
eda_data = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/eda_sample.csv', encoding='latin-1')

In [8]:
test_data

Unnamed: 0.1  Unnamed: 0  prev_id  \
0                0           0      298   
1                1           1     2634   
2                2           2     2404   
3                3           3     5128   
4                4           4     6099   
...            ...         ...      ...   
3945          3945        3945      336   
3946          3946        3946     3521   
3947          3947        3947     2307   
3948          3948        3948     8971   
3949          3949        3949     5065   

                                                  Tweet  HS  Abusive  \
0     USER Ooo camtu.. Biasalah kena seimbang.. Ada ...   0        0   
1     Presiden Minta Sistem Perizinan Berusaha Terin...   0        0   
2     ð???ð???ð???tak berambisi? Mimin bisa aja judu...   0        0   
3     USER Kemaren gubernur pilihan umat impor bawan...   0        0   
4     4. Apakah pernah mengalami kendala dalam menga...   0        0   
...                                                 ...  ..      ...   
3945  Doi tuh native speaker ya.\n\nMother language-...   1        1   
3946  RT USER Kasian ya kacung tetap kacung anies sandi   1        1   
3947  USER kacrut , rank 10 tapi mainnya gk cerdas a...   1        1   
3948  USER Klau kmu di banding AR jauhlah.koen menun...   1        1   
3949  Mana para babi ahok yg ngejek onta, biar di in...   1        1   

      HS_Individual  HS_Group  HS_Religion  HS_Race  HS_Physical  HS_Gender  \
0                 0         0            0        0            0          0   
1                 0         0            0        0            0          0   
2                 0         0            0        0            0          0   
3                 0         0            0        0            0          0   
4                 0         0            0        0            0          0   
...             ...       ...          ...      ...          ...        ...   
3945              1         0            0        0            0          0   
3946              0         1            0        0            0          0   
3947              1         0            0        0            0          0   
3948              1         0            0        0            0          0   
3949              0         1            0        0            0          0   

      HS_Other  HS_Weak  HS_Moderate  HS_Strong  
0            0        0            0          0  
1            0        0            0          0  
2            0        0            0          0  
3            0        0            0          0  
4            0        0            0          0  
...        ...      ...          ...        ...  
3945         1        1            0          0  
3946         1        0            1          0  
3947         1        1            0          0  
3948         1        1            0          0  
3949         1        0            1          0  

[3950 rows x 16 columns]

In [9]:
# Tokenization & Emoticon Processor

regex_str = []
regex_str.append(r'<[^>]+>') # HTML tags
regex_str.append(r'(?:@[\w_]+)') # @-mentions
regex_str.append(r'(?:&[\w_]+)')
regex_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
regex_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
regex_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers
regex_str.append(r"(?:[a-z][a-z'\-_]+[a-z])") # words with - and '
regex_str.append(r'(?:[\w_]+)') # other words
regex_str.append(r'(?:\S)') # anything else

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

def tokenize(sentence):
    return [emoji.demojize(term) for term in tokens_re.findall(sentence)]

In [10]:
# Stopwords Removal & Lemmatizing

lemmatizer = Lemmatizer()
stopword = StopWord()

punctuation = list(string.punctuation)
stops = stopword.get_stopword() + punctuation + ['rt', 'via', '…','•','“', 'user', 'url',
                                                  'xf0', 'x9f', 'x98', 'x84', 'xf0', 'x9f', 'x98', 'x84', 'xf0', 'x9f', 
                                                  'x98', 'x84', 'xf', 'xe2', 'x80', 'x94', 'x82', 'xa4', 'x9d', 'xc2',
                                                  'x86', 'x85', 'x81', 'xa6', 'xa3', 'xad', 'x91', 'x81', 'xe3', 'x99',
                                                  'x', 'a', 'n', 'b',
                                                  '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

def remove_stops_and_lemmatize(tokenized_sentence):
  processed_words = []
  for word in tokenized_sentence:
    if word in kamus_alay.Alay.values:
      row = kamus_alay[kamus_alay['Alay'] == word]
      word = row['Tidak Alay'].values[0]
    if word.lower() not in stops:
      processed_words.append(lemmatizer.lemmatize(word))
  return processed_words

# BERT pytorch

In [11]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("Mendukung")
  print("")
else:
  print("Tidak mendukung GPU hanya CPU")

Mendukung



In [12]:
MAX_SEQ_LEN = 128
  # default max adalah 512, kalau kurang diganti padding
BATCH_SIZE = 2
  # besar batch setiap kali update parameter machine learning
WARMUP_PROPORTION = 0.4
MODEL_NAME = "indolem/indobert-base-uncased"
LEARNING_RATE = 5e-5
EPOCH = 10

In [13]:
transformer = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Prepare Train Data

In [14]:
train_data = pd.concat([
    remaining_data,
    eda_data])

In [15]:
train_data

Unnamed: 0.1  Unnamed: 0  prev_id  \
0              0.0           0      0.0   
1              1.0           1      3.0   
2              2.0           2      6.0   
3              3.0           3      7.0   
4              4.0           4      8.0   
...            ...         ...      ...   
3287           NaN        3287      NaN   
3288           NaN        3288      NaN   
3289           NaN        3289      NaN   
3290           NaN        3290      NaN   
3291           NaN        3291      NaN   

                                                  Tweet  HS  Abusive  \
0     - disaat semua cowok berusaha melacak perhatia...   1        1   
1     Nah admin belanja satu lagi port terbaik nak m...   0        0   
2     Aktor huruhara 98 Prabowo S ingin lengserkan p...   1        0   
3     USER Bu guru enakan jadi jablay atau guru esde...   1        1   
4     USER USER USER USER USER USER Lawan bicara gw ...   1        1   
...                                                 ...  ..      ...   
3287  USER keluarga halilintar hmm masa km gatau. udik'   1        1   
3288  #2019GantiPresiden scra konstitusional hak set...   1        1   
3289  USER si ahok pantasnya dinobatkan sbgai Babi n...   1        1   
3290  YA ALLAH INI XL GAK KURANG KAMPRET LAGI APA KO...   1        1   
3291  USER Den...den...\nNista banget hidup lo..\nUd...   1        1   

      HS_Individual  HS_Group  HS_Religion  HS_Race  HS_Physical  HS_Gender  \
0                 1         0            0        0            0          0   
1                 0         0            0        0            0          0   
2                 1         0            0        0            0          0   
3                 1         0            0        0            0          1   
4                 1         0            1        0            0          0   
...             ...       ...          ...      ...          ...        ...   
3287              1         0            0        0            0          0   
3288              1         0            0        0            0          0   
3289              1         0            0        0            0          0   
3290              0         1            0        0            0          0   
3291              1         0            0        0            0          0   

      HS_Other  HS_Weak  HS_Moderate  HS_Strong  dataset_id  
0            1        1            0          0         NaN  
1            0        0            0          0         NaN  
2            1        1            0          0         NaN  
3            0        1            0          0         NaN  
4            0        1            0          0         NaN  
...        ...      ...          ...        ...         ...  
3287         1        1            0          0      8161.0  
3288         1        1            0          0       123.0  
3289         1        1            0          0      7547.0  
3290         1        0            1          0      9799.0  
3291         1        1            0          0      2277.0  

[9219 rows x 17 columns]

In [16]:
train_docs = []

for i, row in train_data.iterrows():
  tweet = row.Tweet
  if row.HS == 0 and row.Abusive == 0:
    train_docs.append((tweet, "NHS/NA"))
  elif row.HS == 0 and row.Abusive == 1:
    train_docs.append((tweet, "NHS/A"))
  elif row.HS == 1 and row.Abusive == 0:
    train_docs.append((tweet, "HS/NA"))
  elif row.HS == 1 and row.Abusive == 1:
    train_docs.append((tweet, "HS/A"))

In [17]:
len(train_docs)

9219

In [18]:
# valid_data = pd.concat([
#     # test_data,
#     synthetic_test_data])

In [19]:
valid_docs = []

for i, row in synthetic_test_data.iterrows():
  tweet = row.Tweet
  if row.HS == 0 and row.Abusive == 0:
    valid_docs.append((tweet, "NHS/NA"))
  elif row.HS == 0 and row.Abusive == 1:
    valid_docs.append((tweet, "NHS/A"))
  elif row.HS == 1 and row.Abusive == 0:
    valid_docs.append((tweet, "HS/NA"))
  elif row.HS == 1 and row.Abusive == 1:
    valid_docs.append((tweet, "HS/A"))

In [20]:
len(valid_docs)

19750

In [21]:
# text classification dengan 2 label (multilabel)
label_map = {"HS/NA": [1, 0],
             "NHS/A": [0, 1],
             "HS/A": [1, 1],
             "NHS/NA": [0, 0],
            }

## ...

In [22]:
def create_data_loader(instances, label_map = label_map):
  input_ids = []
  input_att_mask = []
  input_label = []
  for (instance, label) in instances:
    encoded_sent = tokenizer.encode(instance, \
                                  add_special_tokens = True, \
                                  max_length = MAX_SEQ_LEN, \
                                  padding = "max_length", \
                                  truncation = True)
    input_ids.append(encoded_sent)
    input_att_mask.append([int(token_id > 0) for token_id in encoded_sent])
    input_label.append(label_map[label])
  
  # ubah ke Torch's Tensor
  input_ids = torch.tensor(input_ids)
  input_att_mask = torch.tensor(input_att_mask)
  input_label = torch.tensor(input_label)

  # gunakan Torch's Dataset
  dataset = TensorDataset(input_ids, input_att_mask, input_label)

  # mengembalikan sebuah abstraksi DataLoader dari Torch
  return DataLoader(dataset,
                    sampler = SequentialSampler(dataset), 
                    #sampler = RandomSampler(dataset), 
                    batch_size = BATCH_SIZE)

In [23]:
train_instances = create_data_loader(train_docs)
valid_instances = create_data_loader(valid_docs)

In [24]:
train_instances

In [25]:
# Classifier adalah layer on top of BERT yang digunakan untuk klasifikasi
# inputnya adalah vektor CLS dari BERT
class Classifier(nn.Module):
    def __init__(self, input_size = 768, output_size = 2, dropout_rate = 0.1):
        super(Classifier, self).__init__()
        layers = []
        layers.append(nn.Linear(input_size, output_size))
        layers.append(nn.LeakyReLU(0.2, inplace = True))
        layers.append(nn.Dropout(dropout_rate))
        layers.append(nn.Softmax(dim = -1))
        self.layers = nn.Sequential(*layers)

    def forward(self, input):
        return self.layers(input)


In [26]:
# jika Anda ingin mengetahui ukuran hidden_size di setiap
# timestep pada BERT / Encoder-nya Transformer
config = AutoConfig.from_pretrained(MODEL_NAME)
hidden_size = int(config.hidden_size)
print(hidden_size)

768


In [27]:
# instansiasi classifier
classifier = Classifier(input_size = config.hidden_size)

In [28]:
# jika GPU available
# Put everything in the GPU if available
if torch.cuda.is_available():    
  classifier.cuda()
  transformer.cuda()

In [29]:
# models parameters
transformer_vars = [p for p in transformer.parameters()]
classifier_vars = [p for p in classifier.parameters()]
all_vars = transformer_vars + classifier_vars

In [30]:
# ada berapa parameter di classifier
# secara teori, ada 768 x 3 + 3 bias
total_vars = 0
for v in classifier_vars:
  print(v)
  total_vars += len(torch.flatten(v))
print("Total parameters di Classifier: ", total_vars)

Parameter containing:
tensor([[ 0.0360,  0.0261,  0.0281,  ..., -0.0134, -0.0230,  0.0278],
        [-0.0155,  0.0053, -0.0237,  ...,  0.0100, -0.0038,  0.0060]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.0001, -0.0018], device='cuda:0', requires_grad=True)
Total parameters di Classifier:  1538


In [ ]:
# training

# optimizer
optimizer = torch.optim.AdamW(all_vars, lr = LEARNING_RATE)

# kita gunakan scheduler untuk warming-up
# Vaswani mengusulkan suatu rumus penentuan learning_rate saat warming-up
# dan setelahnya
num_train_instances = len(train_instances)
num_train_steps = int(num_train_instances / BATCH_SIZE * EPOCH)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

#scheduler = get_constant_schedule_with_warmup(optimizer, 
#                                              num_warmup_steps = num_warmup_steps)

for epoch_i in range(0, EPOCH):
  print(f"Epoch {epoch_i + 1} / {EPOCH}")
  # Reset the total loss for this epoch.
  tr_loss = 0

  # Put the model into training mode.
  transformer.train() 
  classifier.train()

  # Untuk setiap batch di training data
  for step, batch in enumerate(train_instances):
    # print(f">>>> Batch {step + 1}")

    # Unpack sebuah batch dari DataLoader
    batch_input_ids = batch[0].to(device)
    batch_input_att_mask = batch[1].to(device)
    batch_label = batch[2].to(device)

    # Encode real data in the Transformer
    bert_outputs = transformer(batch_input_ids, attention_mask = batch_input_att_mask)
    cls_hidden_states = bert_outputs.pooler_output
    probs = classifier(cls_hidden_states)

    # Categorical Cross-Entropy
      # + (1 - batch_label) * (1 - torch.log(probs))
    loss = -torch.mean(torch.sum(batch_label * torch.log(probs), dim = 1))

    # Agar gradient tidak menumpuk; ini diperlukan sebelum
    # menghitung gradient dengan loss.backward()
    optimizer.zero_grad()

    # hitung gradient
    loss.backward()

    # update parameter
    optimizer.step()

    # akumulasi loss dalam 1 epoch
    tr_loss += loss.item()

    # update learning rate
    #scheduler.step()

  # rata-rata loss dalam 1 epoch
  avg_loss = tr_loss / len(train_instances)

  print(f"Average loss: {avg_loss}")

  ### Uji coba di Validation Data
  print("Uji coba di Validation Data ...")

  # model eval, agar layer seperti dropout yang menghasilkan
  # sesuai yang random tidak digunakan
  classifier.eval()
  transformer.eval()

  val_loss = 0
  pred_labels = []
  true_labels = []

  # Untuk setiap batch di validation data
  for step, batch in enumerate(valid_instances):

    # Unpack sebuah batch dari DataLoader
    batch_input_ids = batch[0].to(device)
    batch_input_att_mask = batch[1].to(device)
    batch_label = batch[2].to(device)

    # jangan track gradient! ini sedang evaluasi, bukan training
    with torch.no_grad(): 
      bert_outputs = transformer(batch_input_ids, attention_mask = batch_input_att_mask)
      cls_hidden_states = bert_outputs.pooler_output
      probs = classifier(cls_hidden_states)

      # Categorical Cross-Entropy
      loss = -torch.mean(torch.sum(batch_label * torch.log(probs), dim = 1))

      # akumulasi loss di validation set
      val_loss += loss.item()

    # akumulasi prediksi
    _, pred_l = torch.max(probs, dim = 1) # outnya adalah 2-tuple: (max, max_indices)
    pred_labels += pred_l.detach().cpu()
    _, true_l = torch.max(batch_label, dim = 1)
    true_labels += true_l.detach().cpu()

  # average loss
  avg_val_loss = val_loss / len(valid_instances)

  # accuracy
  pred_labels = torch.stack(pred_labels).numpy()
  true_labels = torch.stack(true_labels).numpy()
  val_accuracy = np.sum(pred_labels == true_labels) / len(pred_labels)
  print(f"Average loss di validation data: {avg_val_loss}")
  print(f"Accuracy di validation data: {val_accuracy}")

  print("")

Epoch 1 / 10


In [ ]:
from google.colab import files

torch.save(transformer.state_dict(), 'checkpoint.pth')
# download checkpoint file
files.download('checkpoint.pth')